In [23]:
import json as json
import pprint as pp
from opensearchpy import OpenSearch
import os
import pickle
import search
import index
import recipe_parser as rp
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

## Index Setup

In [24]:
host = 'api.novasearch.org'
port = 443
user = 'user201' # Add your user name here.
password = 'Lrr1531' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

### Index Deletion


In [32]:
index.delete_index(client, index_name)

{'acknowledged': True}


### Index Creation

In [26]:
index.create_index(client, index_name)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user201'}


## Recipes Setup

In [11]:
# check if a pickle file exists

if os.path.exists('./pickle_files/recipe_titles.pkl'):
    titles = pickle.load(open('./pickle_files/recipe_titles.pkl', 'rb'))
else:
    titles = rp.get_recipe_titles()

if os.path.exists('./pickle_files/recipe_descs.pkl'):
    descs = pickle.load(open('./pickle_files/recipe_descs.pkl', 'rb'))
else:
    descs = rp.get_recipe_descs()



### Index Recipes

In [6]:
index.index_document(client, index_name, rp.get_recipes())

created: How To Make Chicken Parmesan
created: How to Make Pesto
created: How To Make Corn Tortillas From Scratch
created: How To Make Elote (Mexican Street Corn)
created: How To Make Macarons
created: How To Make Meringue
created: How To Make Handmade Pasta
created: How to Make Perfect Polenta
created: How To Make Miso Soup
created: How to Make Perfect Guacamole
created: How to Cook Trout
created: How to Cook a Turkey
created: How to Cook: Boiled Eggs
created: How To Cook A Perfect Risotto
created: How to Cook: Crispy Tofu
created: How to Cook: Brown Rice
created: How To Cook Salmon in the Oven
created: How To Cook Brown Rice
created: How To Cook Filet Mignon
created: How To Cook Lentils on the Stove
created: Sylvia's World Famous Talked About Spareribs
created: Holiday Eggnog
created: Holiday Pizza
created: Holiday Salad
created: Holiday Seafood Pot
created: Holiday Chicken Salad
created: Holiday milk punch
created: Holiday Manhattan
created: Snickerdoodles I
created: Holiday Pumpkin

# Text-based Search

### Simple query search and response containing title and description

In [8]:
query = "carrot"
search.search_titleTxt(client, index_name, query)

Found the following recipes:
'Carrot Hot Dogs'
'Light Carrot Cake'
'Carrot Cake Recipe'
'Carrot Cake for Two'


### Search Recipes with duration

In [9]:
recipe = "Doughnut"
search.search_titleTotalTime(client, index_name, recipe)

Found the following recipes:
'Doughnut Tree - Total time: 45 minutes'

'Doughnut Strawberry Shortcake - Total time: 10 minutes'



### Search Recipes by duration

In [10]:
max_time = 60
search.search_recipeByTime(client, index_name, max_time)

Found the following recipes:
'Chinese Takeout-Style Sweet and Sour Spare Ribs - Total time: 60 minutes'

'Sweet and Sour Chicken - Total time: 30 minutes'

'Sweet and Sour Chicken Tenders - Total time: 55 minutes'

("Sukiyaki in an American Kitchen from 'Hiroko's American Kitchen' - Total "
 'time: 60 minutes')

'No Recipe Recipe: Simple Egg Drop Soup - Total time: 12 minutes'



### Text-Based Search using term queries

In [11]:
query = "chicken"
search.search_titleTxt_terms(client, index_name, query)

Found the following recipes:
('Grilled Cilantro Lime Chicken - Containing the termSkinless boneless chicken '
 'breasts marinated with lime and cilantro and grilled. ')

('How to Make Homemade Pasta (Without a Pasta Maker!) - Containing the termI '
 'can’t remember the last time I made chicken stock in anything but a pressure '
 'cooker. It’s fast, it’s convenient, and the stock itself is superb. Lots of '
 'us have chicken bones from rotisserie chickens or from cutting up '
 'chicken&amp;hellip;')

('Roll-Your-Own Burritos - Containing the termPerfect for families with '
 'different tastes (i.e. picky eaters), each person makes their dinner exactly '
 'the way they want it. A prepared rotisserie chicken makes adding chicken to '
 'this recipe a snap.')

('Sweet and Sour Chicken - Containing the termChinese-style sweet and sour '
 'chicken, stir-fried with bell peppers and pineapple chunks.')

('Minestrone Soup - Containing the termMinestrone soup is an Italian classic! '
 'This versio

### Text-Based Search using boolean queries


In [12]:
query = "chocolate"
search.search_titleIngredients_bool(client, index_name, query)

Found the following recipes:
('Chocolate Chocolate Chip Cookies I - Containing the following ingredients:  '
 "['butter', 'sugar', 'egg', 'vanilla extract', 'flour', 'cocoa powder', "
 "'baking soda', 'salt', 'chocolate', 'walnuts']")

("Chocolate Chip Cookies - Containing the following ingredients:  ['butter', "
 "'dark brown sugar', 'sugar', 'egg', 'vanilla extract', 'chocolate', 'flour', "
 "'baking soda', 'salt']")

('No-Bake Chocolate Pudding Cake - Containing the following ingredients:  '
 "['chocolate hazelnut spread', 'graham cracker', 'instant chocolate pudding', "
 "'milk', 'cream cheese', 'sugar', 'vanilla extract', 'chocolate', 'chocolate "
 "frosting', 'raspberries', 'graham cracker crumbs', 'chocolate shaving']")

("Entenmann's Chocolate Chip Loaf Cake - Containing the following "
 "ingredients:  [None, 'flour', 'salt', 'butter', 'sugar', 'vanilla extract', "
 "'egg', 'chocolate', None, None, 'chocolate', None, 'vanilla extract', None]")

('Chocolate-Covered Strawberry Br

# Encoding - Dual Encoders

In [3]:
#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")





### Index the embeddings

In [14]:
titles_emb = encode(titles)


with open('./pickle_files/title_embeddings.pickle', 'wb') as f:
    pickle.dump(titles_emb, f)
    

index.index_titleEmbeddings(client, index_name, titles)


created: How To Make Chicken Parmesan
created: How to Make Pesto
created: How To Make Corn Tortillas From Scratch
created: How To Make Elote (Mexican Street Corn)
created: How To Make Macarons
created: How To Make Meringue
created: How To Make Handmade Pasta
created: How to Make Perfect Polenta
created: How To Make Miso Soup
created: How to Make Perfect Guacamole
created: How to Cook Trout
created: How to Cook a Turkey
created: How to Cook: Boiled Eggs
created: How To Cook A Perfect Risotto
created: How to Cook: Crispy Tofu
created: How to Cook: Brown Rice
created: How To Cook Salmon in the Oven
created: How To Cook Brown Rice
created: How To Cook Filet Mignon
created: How To Cook Lentils on the Stove
created: Sylvia's World Famous Talked About Spareribs
created: Holiday Eggnog
created: Holiday Pizza
created: Holiday Salad
created: Holiday Seafood Pot
created: Holiday Chicken Salad
created: Holiday milk punch
created: Holiday Manhattan
created: Snickerdoodles I
created: Holiday Pumpkin

In [5]:
descs_emb = encode(descs)

with open('./pickle_files/desc_embeddings.pickle', 'wb') as f:
   pickle.dump(descs_emb, f)

index.index_descEmbeddings(client, index_name, descs)

created: Master the classic dish of chicken Parmesan by starting with the chicken, choosing a marinara sauce you love, and using a trio of cheese. 
created: Spread it on sandwiches, toss it with pasta, or treat yourself a single happy spoonful, but definitely absolutely positively make pesto any chance you get.
created: Corn tortillas are made with just two ingredients: masa harina and water. What could be easier? Here's how to make tortillas for the best-ever tacos!
created: SAJ86VE751
created: SN2KP4SC4N
created: With a crisp outer shell, slightly chewy center, and a subtle sweetness, baked meringue is a melt-in-your-mouth delight.
created: JDNP6PW4XM
created: U5QJHLA422
created: H5KN02RMQ1
created: The BEST guacamole! So easy to make with ripe avocados, salt, serrano chiles, cilantro and lime. Garnish with red radishes or jicama. Serve with tortilla chips. Watch how to make guacamole - it's easy!
created: DZTL32VJHX
created: 91VYDHY2F2
created: Nobody likes overcooked, rubbery break

### Embedding Title Search

### Title embedding

In [6]:
query = "cake"
query_emb = encode(query)

search.search_titleEmbedding(client, index_name, query_emb)

Found the following recipes:

'My Birthday Cake'

"A Chocolate Cake That's Got It All"

'Christmas Cake Cookies'

'Chocolate Chip Cookie Cake'

'Red Velvet Cake'



### Description embedding

In [4]:
query = "chicken marsala"
query_emb = encode(query)

search.search_title_descEmbedding(client, index_name, query_emb)

Found the following recipes:

('Title of recipe: Our step-by-step recipe for classic chicken Marsala, a '
 'delicious yet surprisingly easy one-pot chicken dinner with all the Italian '
 'flavor you crave. ')

('Description of recipe: Our step-by-step recipe for classic chicken Marsala, '
 'a delicious yet surprisingly easy one-pot chicken dinner with all the '
 'Italian flavor you crave. ')

('Description of recipe: Our step-by-step recipe for classic chicken Marsala, '
 'a delicious yet surprisingly easy one-pot chicken dinner with all the '
 'Italian flavor you crave. ')

('Title of recipe: Chinese-style sweet and sour chicken, stir-fried with bell '
 'peppers and pineapple chunks.')

('Description of recipe: Chinese-style sweet and sour chicken, stir-fried with '
 'bell peppers and pineapple chunks.')

